In [5]:
pip install sko requests

                                              0.0/62.6 kB ? eta -:--:--
     ------------------------               41.0/62.6 kB 960.0 kB/s eta 0:00:01
     -------------------------------------- 62.6/62.6 kB 670.9 kB/s eta 0:00:00
                                              0.0/96.6 kB ? eta -:--:--
     ---------------------------------------- 96.6/96.6 kB 2.8 MB/s eta 0:00:00
                                              0.0/61.5 kB ? eta -:--:--
     ---------------------------------------- 61.5/61.5 kB ? eta 0:00:00
                                              0.0/123.6 kB ? eta -:--:--
     -------------------------------------- 123.6/123.6 kB 7.1 MB/s eta 0:00:00
                                              0.0/157.0 kB ? eta -:--:--
     -------------------------------------- 157.0/157.0 kB 9.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
import pandas as pd
import requests
import datetime
from sko.SA import SA_TSP

import warnings
warnings.filterwarnings('ignore')

In [9]:
def loadProc (i, i_fin, isFixed = False):
    proc = i * 100 / i_fin
    int_proc = round(proc, 1)
    if int_proc % 1 == 0 and isFixed == False:
        print('['+ '='*int(int_proc)+ ' '*(100-int(int_proc))+ ']', int(int_proc), ' %', end='\r')
        isFixed = True
    if int_proc % 1 != 0 and isFixed == True:
        isFixed = False

def distance_matrix(route_id, df, X_coord_col, Y_coord_col, Route_index_col):
    df_coordinates = df[[Y_coord_col, X_coord_col]][df[Route_index_col]==route_id]
    k = df_coordinates.index.values.tolist()
    coordinates_string = ""
    for i in k:
        if i==k[-1]:
            coordinates_string = coordinates_string + str(df_coordinates[Y_coord_col][i]) + "," + str(df_coordinates[X_coord_col][i])
        else:
            coordinates_string = coordinates_string + str(df_coordinates[Y_coord_col][i]) + "," + str(df_coordinates[X_coord_col][i]) + ";"
    location_req = requests.get(f'http://router.project-osrm.org/table/v1/driving/{coordinates_string}?annotations=duration,distance')
    distances = location_req.json()["distances"]
    durations = location_req.json()["durations"]

    return distances, durations

def find_route_SA(weighted_matrix, start_point=0, view_distance=False):
    num_points = np.array(weighted_matrix).shape[0]
    distance_matrix = np.array(weighted_matrix)

    def cal_total_distance(routine):
        '''The objective function. input routine, return total distance.
        cal_total_distance(np.arange(num_points))
        '''
        num_points, = routine.shape
        return sum(
            [distance_matrix[routine[i % num_points], routine[(i + 1) % num_points]] for i in range(num_points)])

    sa_tsp = SA_TSP(func=cal_total_distance, x0=range(num_points), T_max=100, T_min=1, L=10 * num_points)
    order_list, best_distance = sa_tsp.run()
    order_list = np.roll(order_list, -1 * np.where(order_list == 0)[0][0])

    if view_distance:
        return order_list, best_distance
    else:
        return order_list

def order_route(order_list, df):
    df_route_ordered = df.reindex(order_list)
    df_route_ordered = df_route_ordered.reset_index(drop=True, inplace=False)
    return df_route_ordered

In [10]:

def run_algorithm(df_c, days_ago, df_final):

    # slice the dataframe to get the last days_ago days
    df_c["Entry_Date"] = pd.to_datetime(df_c["Entry_Date"])
    datetime_now = datetime.datetime.now()
    first_date = datetime_now - datetime.timedelta(days=days_ago, hours=datetime_now.hour, minutes=datetime_now.minute, seconds=datetime_now.second + 1)
    df_c = df_c[df_c["Entry_Date"]>=first_date]

    # slice the dataframe to get only the last 31 days
    # df_c["Entry_Date"] = pd.to_datetime(df_c["Entry_Date"])
    # datetime_now = datetime.datetime(2023, 5, 31, 23, 59, 59)
    # first_date = datetime_now - datetime.timedelta(days=31, hours=datetime_now.hour, minutes=datetime_now.minute, seconds=datetime_now.second)
    # df_c = df_c[df_c["Entry_Date"]>=first_date]
    # df_c = df_c[df_c["Entry_Date"]<=datetime_now]

    routes_index = df_c["RouteIndex"].unique()

    df_c["OldInterPointsDistance"] = None
    df_c["NewInterPointsDistance"] = None

    df_c["OldInterPointsDuration"] = None
    df_c["NewInterPointsDuration"] = None

    df_c["OptimizedIndex"] = None

    skiped = []
    ind = 0

    for route_index in routes_index:
        print("Route index: ", route_index , "  ", ind, " of ", len(routes_index), end="\r")
        try:
            if len(df_c[df_c["RouteIndex"] == route_index]) <= 1:
                skiped.append(route_index)
                continue
            else:
                dist_matrix, time_matrix = distance_matrix(route_index, df_c, "X_Coardinate", "Y_Coardinate", "RouteIndex")
                
                if len(df_c[df_c["RouteIndex"] == route_index]) > 2:
                    ordered_list, distance = find_route_SA(dist_matrix, view_distance=True)
                else:
                    ordered_list = [0, 1]

                distances_old = []
                indexes_1 = [index for index in df_c["OrderIndex"][df_c["RouteIndex"] == route_index]]
                for i in range(len(indexes_1)-1):
                    distances_old.append(dist_matrix[indexes_1[i]][indexes_1[i+1]])
                distances_old.append(dist_matrix[indexes_1[-1]][indexes_1[0]])

                distances_new = []
                indexes_2 = [index for index in ordered_list]
                for i in range(len(indexes_2)-1):
                    distances_new.append(dist_matrix[indexes_2[i]][indexes_2[i+1]])
                distances_new.append(dist_matrix[indexes_2[-1]][indexes_2[0]])

                df_c["OldInterPointsDistance"][df_c["RouteIndex"] == route_index] = distances_old
                df_c["NewInterPointsDistance"][df_c["RouteIndex"] == route_index] = distances_new

                duration_old = []
                for i in range(len(indexes_1)-1):
                    duration_old.append(time_matrix[indexes_1[i]][indexes_1[i+1]])
                duration_old.append(time_matrix[indexes_1[-1]][indexes_1[0]])

                duration_new = []
                for i in range(len(indexes_2)-1):
                    duration_new.append(time_matrix[indexes_2[i]][indexes_2[i+1]])
                duration_new.append(time_matrix[indexes_2[-1]][indexes_2[0]])

                df_c["OldInterPointsDuration"][df_c["RouteIndex"] == route_index] = duration_old
                df_c["NewInterPointsDuration"][df_c["RouteIndex"] == route_index] = duration_new

                df_new = df_c[df_c["RouteIndex"] == route_index].iloc[ordered_list]
                df_new["OptimizedIndex"] = list(range(len(ordered_list)))

                df_final = pd.concat([df_final, df_new])
                
        except Exception as e:
            print('\n', e, "Route index: ", route_index)
            skiped.append(route_index)
            pass
        ind += 1

    return df_final
        
        # print("Route" + str(route_index), end='\r')


In [11]:
df_origin = pd.read_csv("AppendVISIT-START-august.csv").sort_values("TrueIndex")

In [12]:
df_origin

,Customer_Code,Customer_Name,Entry_Date,Entry_Time,InPlace_NotInPlace,InRoute_OutofRoute,OrderIndex,RouteIndex,SR_Code,SR_Name,TrueIndex,X_Coardinate,Y_Coardinate
45462,44066606033,Home,2023-08-01 00:00:00.000,1899-12-30 00:00:00.000,In Place,In Route,0,227,8,Stanislav Berzan,6864,47.168536,29.195862
20567,17870,DENIADI SRL,2023-08-01 00:00:00.000,1899-12-30 09:31:41.000,In Place,In Route,1,227,8,Stanislav Berzan,6865,46.989949,28.884396
1395,3775,Silviu Dan SRL,2023-08-01 00:00:00.000,1899-12-30 10:08:30.000,In Place,In Route,2,227,8,Stanislav Berzan,6866,46.988718,28.886459
35569,23668,Rompetrol Moldova SA,2023-08-01 00:00:00.000,1899-12-30 10:17:15.000,In Place,In Route,3,227,8,Stanislav Berzan,6867,46.986643,28.883750
31559,22374,MarChi Group SRL,2023-08-01 00:00:00.000,1899-12-30 10:26:42.000,In Place,In Route,4,227,8,Stanislav Berzan,6868,46.988307,28.882031
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34598,23393,Moldretail Group SRL (Fidesco),2023-08-31 00:00:00.000,1899-12-30 13:58:44.000,In Place,In Route,34,15636,227,Danu Insuratelu,516016,47.249260,28.317330
25100,20170,SCLIFOS TATIANA II,2023-08-31 00:00:00.000,1899-12-30 15:34:17.000,In Place,Out of Route,35,15636,227,Danu Insuratelu,516017,47.247746,28.322233
25192,20193,Raf Nicolai SRL,2023-08-31 00:00:00.000,1899-12-30 15:47:13.000,In Place,Out of Route,36,15636,227,Danu Insuratelu,516018,47.249763,28.314952
24998,20150,Moldretail Group SRL,2023-08-31 00:00:00.000,1899-12-30 16:00:41.000,In Place,In Route,37,15636,227,Danu Insuratelu,516019,47.248631,28.313031


In [13]:
# create an empty dataframe
df_final = pd.DataFrame(columns=df_origin.columns)

In [14]:
df_c = df_origin.copy()
days_ago = 9999
df_fin = run_algorithm(df_c, days_ago, df_final)

In [15]:
df_fin.to_csv("AppendVISIT-START-optimized-august.csv", index=False)

In [16]:
df_final

,Customer_Code,Customer_Name,Entry_Date,Entry_Time,InPlace_NotInPlace,InRoute_OutofRoute,OrderIndex,RouteIndex,SR_Code,SR_Name,TrueIndex,X_Coardinate,Y_Coardinate


In [65]:
df_time = pd.read_csv("AppendVISIT-START-optimized.csv")

In [66]:
df_time

,SR_Code,SR_Name,Customer_Code,Customer_Name,Entry_Date,Entry_Time,InPlace_NotInPlace,InRoute_OutofRoute,X_Coardinate,Y_Coardinate,RouteIndex,OrderIndex,TrueIndex,OldInterPointsDistance,NewInterPointsDistance,OptimizedIndex
0,11,Denis Balan,44066606033,Home,2023-05-01,1899-12-30 00:00:00.000,In Place,In Route,47.119900,29.118730,0,0,0,28899.2,28826.0,0.0
1,11,Denis Balan,23186,Sabra Food SRL,2023-05-01,1899-12-30 08:52:14.000,In Place,In Route,47.000729,28.859768,0,5,5,767.4,503.4,1.0
2,11,Denis Balan,26742,Vlama 15 SRL (Star Kebab)( CS ),2023-05-01,1899-12-30 08:45:00.000,In Place,In Route,47.000064,28.859884,0,1,1,29.8,73.2,2.0
3,11,Denis Balan,23034,RESTOMAMA,2023-05-01,1899-12-30 08:46:50.000,In Place,In Route,46.999851,28.859797,0,3,3,453.6,5.7,3.0
4,11,Denis Balan,22574,Vlama 15 SRL (Star Kebab),2023-05-01,1899-12-30 08:45:04.000,In Place,In Route,46.999794,28.859905,0,2,2,5.7,24.1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46516,9,Ion Gulea,4006,CLIP SRL,2023-05-30,1899-12-30 16:53:50.000,In Place,In Route,46.954911,28.770059,2127,27,69005,72.0,85.9,34.0
46517,9,Ion Gulea,15888,Lukoil Moldova SRL,2023-05-30,1899-12-30 16:39:05.000,In Place,In Route,46.971341,28.765234,2127,26,69004,1964.3,2059.4,35.0
46518,9,Ion Gulea,23119,AFLOGEN SRL,2023-05-30,1899-12-30 15:52:10.000,In Place,In Route,46.962838,28.754391,2127,22,69000,216.6,1103.2,36.0
46519,9,Ion Gulea,20775,SIRBU-TEIUL II,2023-05-30,1899-12-30 16:10:32.000,In Place,In Route,46.961407,28.752509,2127,23,69001,205.4,317.7,37.0


In [67]:
df_c = df_time.copy().sort_values(by=['RouteIndex', 'OrderIndex'])
df_c_2 = df_time.copy()
df_c_s = df_c_2.copy().sort_values(by=['RouteIndex', 'OptimizedIndex'])

In [68]:
df_c_s

,SR_Code,SR_Name,Customer_Code,Customer_Name,Entry_Date,Entry_Time,InPlace_NotInPlace,InRoute_OutofRoute,X_Coardinate,Y_Coardinate,RouteIndex,OrderIndex,TrueIndex,OldInterPointsDistance,NewInterPointsDistance,OptimizedIndex
0,11,Denis Balan,44066606033,Home,2023-05-01,1899-12-30 00:00:00.000,In Place,In Route,47.119900,29.118730,0,0,0,28899.2,28826.0,0.0
1,11,Denis Balan,23186,Sabra Food SRL,2023-05-01,1899-12-30 08:52:14.000,In Place,In Route,47.000729,28.859768,0,5,5,767.4,503.4,1.0
2,11,Denis Balan,26742,Vlama 15 SRL (Star Kebab)( CS ),2023-05-01,1899-12-30 08:45:00.000,In Place,In Route,47.000064,28.859884,0,1,1,29.8,73.2,2.0
3,11,Denis Balan,23034,RESTOMAMA,2023-05-01,1899-12-30 08:46:50.000,In Place,In Route,46.999851,28.859797,0,3,3,453.6,5.7,3.0
4,11,Denis Balan,22574,Vlama 15 SRL (Star Kebab),2023-05-01,1899-12-30 08:45:04.000,In Place,In Route,46.999794,28.859905,0,2,2,5.7,24.1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46516,9,Ion Gulea,4006,CLIP SRL,2023-05-30,1899-12-30 16:53:50.000,In Place,In Route,46.954911,28.770059,2127,27,69005,72.0,85.9,34.0
46517,9,Ion Gulea,15888,Lukoil Moldova SRL,2023-05-30,1899-12-30 16:39:05.000,In Place,In Route,46.971341,28.765234,2127,26,69004,1964.3,2059.4,35.0
46518,9,Ion Gulea,23119,AFLOGEN SRL,2023-05-30,1899-12-30 15:52:10.000,In Place,In Route,46.962838,28.754391,2127,22,69000,216.6,1103.2,36.0
46519,9,Ion Gulea,20775,SIRBU-TEIUL II,2023-05-30,1899-12-30 16:10:32.000,In Place,In Route,46.961407,28.752509,2127,23,69001,205.4,317.7,37.0


In [69]:
# df_final = pd.DataFrame(columns=df_c.columns)
routes_index = df_c["RouteIndex"].unique()

skiped = []
duration_list_old = []
duration_list_new = []
ind = 0

for route_index in routes_index:
    print("Route index: ", route_index , "  ", ind, " of ", len(routes_index - 1), end="\r")
    try:
        if len(df_c[df_c["RouteIndex"] == route_index]) <= 1:
            skiped.append(route_index)
            continue
        else:
            dist_matrix, time_matrix = distance_matrix(route_index, df_c, "X_Coardinate", "Y_Coardinate", "RouteIndex")

            duration_old = []
            indexes_1 = [index for index in df_c["OrderIndex"][df_c["RouteIndex"] == route_index]]
            for i in range(len(indexes_1)-1):
                duration_old.append(time_matrix[indexes_1[i]][indexes_1[i+1]])
            duration_old.append(time_matrix[indexes_1[-1]][indexes_1[0]])

            duration_new = []
            indexes_2 = [index for index in df_c_s["OrderIndex"][df_c_s["RouteIndex"] == route_index]]
            for i in range(len(indexes_2)-1):
                duration_new.append(time_matrix[indexes_2[i]][indexes_2[i+1]])
            duration_new.append(time_matrix[indexes_2[-1]][indexes_2[0]])

            for el in duration_old: duration_list_old.append(el)
            for el in duration_new: duration_list_new.append(el)

    except Exception as e:
        print('\n', e, "Route index: ", route_index)
        skiped.append(route_index)
        pass
    ind += 1



In [70]:
df_end_time = df_time.copy().sort_values(by=['RouteIndex', 'OrderIndex'])
df_end_time["OldInterPointsDuration"] = duration_list_old
df_end_time["NewInterPointsDuration"] = duration_list_new

In [71]:
len(df_c)

46521

In [72]:
print(indexes_1)
print(indexes_2)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
[0, 25, 21, 20, 19, 18, 29, 31, 38, 34, 35, 36, 17, 11, 15, 13, 14, 12, 16, 10, 8, 9, 7, 5, 3, 4, 6, 1, 2, 37, 32, 33, 30, 28, 27, 26, 22, 23, 24]


In [73]:
print(dist_matrix)

[[0, 22895.4, 22981.3, 21269, 21410.3, 21482.9, 21691.5, 22487.7, 23935.7, 23545, 17672, 11420.2, 19573.3, 13434.6, 13175.5, 12472.2, 11792.8, 10875, 9953.8, 9899.2, 9701.9, 12953.5, 15247.7, 12076.2, 12281.6, 8670.9, 13756.4, 10743.4, 10710, 10427.6, 10879.1, 10332.4, 10628.1, 10600.1, 10597.2, 10786.3, 11088.9, 12197.7, 10536.5], [19802, 0, 85.9, 2526.9, 2232.6, 2225, 2059.4, 1143.2, 1258.1, 867.5, 1393.2, 6133.5, 13639, 7500.2, 7241.1, 6537.8, 5760.9, 12712.9, 13175.3, 12172.5, 12603.2, 12328, 10729.1, 10933.1, 11138.6, 14009.3, 9532.1, 11395.3, 11391.5, 11696.2, 11753.1, 11730.4, 12122.9, 12094.9, 12265.9, 12455, 12591.2, 6608.2, 11942.4], [19978.6, 176.6, 0, 2703.5, 2409.2, 2401.7, 2236.1, 1319.9, 1434.7, 1044.1, 1326.6, 6066.8, 13572.3, 7433.6, 7174.5, 6471.2, 5694.2, 6643.7, 13351.9, 12349.1, 12779.9, 12504.6, 10905.8, 11109.8, 11315.2, 14186, 9708.7, 11572, 11568.2, 11872.8, 11929.8, 11907.1, 12299.5, 12271.6, 12442.5, 12631.6, 6674.4, 6541.5, 12119.1], [18253.2, 2526.9, 2612.8

In [74]:
print(time_matrix)

[[0, 1489.9, 1500.4, 1348.3, 1364.7, 1400.6, 1395.7, 1455.4, 1676.3, 1585.3, 1304.5, 766.4, 1300.2, 925.2, 881.1, 835, 798.7, 728.4, 679.8, 672.2, 659.8, 710.3, 824, 611.2, 626.3, 577.4, 732.3, 776.9, 769.9, 742.9, 780.6, 727.9, 741.8, 740.3, 720.7, 755.3, 757.8, 920.3, 724], [1454.4, 0, 10.5, 238.2, 199.7, 209.9, 183.9, 108, 198.9, 107.9, 135.7, 844.4, 1336.5, 961.5, 917.4, 871.3, 815, 1102.6, 1088, 1078.9, 1028.9, 1035.6, 877.7, 885.5, 900.6, 1102.8, 832.5, 1004.5, 995.3, 1027.8, 1057.7, 1020.8, 1077.3, 1075.8, 1066.5, 1101.1, 1089.5, 953.9, 1044.1], [1487, 32.6, 0, 270.8, 232.3, 242.5, 216.5, 140.6, 231.5, 140.5, 138.8, 847.5, 1339.6, 964.6, 920.5, 874.4, 818.1, 888.9, 1120.6, 1111.5, 1061.5, 1068.2, 910.3, 918.1, 933.2, 1135.4, 865.1, 1037.1, 1027.9, 1060.4, 1090.3, 1053.4, 1109.9, 1108.4, 1099.1, 1133.7, 891.5, 957, 1076.7], [1322.9, 236.5, 247, 0, 70.9, 83.4, 80.3, 140, 356.2, 265.2, 372.2, 1000.6, 1534.4, 1159.4, 1115.3, 1069.2, 1032.9, 971.1, 956.5, 947.4, 897.4, 904.1, 746.2, 

In [75]:
duration_list_new

[2465.1,
 6.7,
 2.2,
 0.5,
 100.2,
 60.4,
 10.2,
 9.3,
 89.5,
 46.4,
 63.6,
 133.5,
 16.5,
 12.6,
 0.6,
 44.8,
 65.2,
 37.8,
 125.1,
 114.8,
 36.1,
 51.8,
 28.7,
 76.6,
 113.9,
 8.9,
 97.7,
 0,
 1.4,
 4.2,
 35.4,
 11.8,
 18.9,
 109.5,
 87.3,
 120.5,
 142.6,
 192.1,
 187.5,
 29.3,
 123.9,
 1.3,
 480.9,
 2200.5,
 2192.4,
 443.8,
 270.9,
 248.4,
 185.5,
 113.2,
 0,
 74,
 45.2,
 12.2,
 61.1,
 8.5,
 52.6,
 1.4,
 136.8,
 82.7,
 23.9,
 105.9,
 132.4,
 34.6,
 46.8,
 28.6,
 0,
 41.4,
 19.3,
 84,
 243.8,
 57.8,
 19.1,
 61,
 154.4,
 61.7,
 24.9,
 2513.6,
 2480.3,
 81.9,
 60.9,
 1.2,
 98,
 80.5,
 100.1,
 132.4,
 58.6,
 58.6,
 31.2,
 11.5,
 178.4,
 12.4,
 106.9,
 29.1,
 109.8,
 3.2,
 6.3,
 69.9,
 146.4,
 249.2,
 68.6,
 19.8,
 44.5,
 0,
 38.1,
 103.5,
 141.3,
 102.5,
 77.2,
 0.5,
 79.6,
 32.1,
 46.8,
 37.8,
 39.6,
 38.5,
 2457.8,
 2543.5,
 100.2,
 2.7,
 72,
 0,
 24.5,
 35.7,
 19.3,
 79.7,
 190.2,
 27.6,
 1.3,
 41.8,
 188.4,
 8.9,
 30.2,
 80.7,
 125.9,
 173.7,
 0.7,
 30.5,
 112.3,
 41.1,
 116.9,
 103

In [76]:
df_end_time[df_end_time["RouteIndex"]==route_index]

,SR_Code,SR_Name,Customer_Code,Customer_Name,Entry_Date,Entry_Time,InPlace_NotInPlace,InRoute_OutofRoute,X_Coardinate,Y_Coardinate,RouteIndex,OrderIndex,TrueIndex,OldInterPointsDistance,NewInterPointsDistance,OptimizedIndex,OldInterPointsDuration,NewInterPointsDuration
46482,9,Ion Gulea,44066606033,Home,2023-05-30,1899-12-30 00:00:00.000,In Place,In Route,46.994006,28.672187,2127,0,68978,22895.4,8670.9,0.0,1489.9,577.4
46509,9,Ion Gulea,23677,Vento Retail SRL,2023-05-30,1899-12-30 07:50:36.000,In Place,In Route,46.958490,28.832100,2127,1,68979,85.9,1349.6,27.0,10.5,170.7
46510,9,Ion Gulea,20751,BIAL SRL,2023-05-30,1899-12-30 08:10:29.000,In Place,Out of Route,46.957897,28.831652,2127,2,68980,2703.5,577.4,28.0,270.8,83.5
46506,9,Ion Gulea,17318,DAFNACOM S.R.L.,2023-05-30,1899-12-30 08:51:16.000,In Place,In Route,46.979387,28.835798,2127,3,68981,358.1,517.5,24.0,70.9,51.5
46507,9,Ion Gulea,3522,SISCANUSCA,2023-05-30,1899-12-30 09:24:06.000,In Place,In Route,46.977532,28.836645,2127,4,68982,122.1,63.4,25.0,25.7,16.1
46505,9,Ion Gulea,3576,CGG Comendant SRL,2023-05-30,1899-12-30 09:38:53.000,In Place,In Route,46.977330,28.835967,2127,5,68983,307.1,527.6,23.0,50.4,89.3
46508,9,Ion Gulea,3514,Neo Slavir SRL,2023-05-30,1899-12-30 09:47:57.000,In Place,In Route,46.975861,28.834960,2127,6,68984,937.7,95.2,26.0,84.5,18.3
46504,9,Ion Gulea,23719,Rompetrol Moldova SA,2023-05-30,1899-12-30 09:59:49.000,In Place,In Route,46.968049,28.833337,2127,7,68985,1596.6,212.0,22.0,226.0,23.3
46502,9,Ion Gulea,3577,CGG Comendant SRL,2023-05-30,1899-12-30 10:31:06.000,In Place,In Route,46.955072,28.836543,2127,8,68986,390.6,359.1,20.0,91.2,35.2
46503,9,Ion Gulea,14678,GLK-MAGIC SRL,2023-05-30,1899-12-30 10:37:31.000,In Place,In Route,46.957783,28.836975,2127,9,68987,1386.5,217.3,21.0,270.0,45.5


In [77]:
df_end_time.to_csv("AppendVISIT-START-TIME-optimized.csv", index=False)